In [58]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics
from sklearn import linear_model
from sklearn import ensemble
from sklearn import tree
from sklearn import svm
import xgboost

In [59]:
data = pd.read_csv(r"C:\Users\prana\Documents\Rainfall ML Project\Rainfall.csv")

In [60]:
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Delhi,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Delhi,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Delhi,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Delhi,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Delhi,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [61]:
data.isnull().sum() #Evaporation    Sunshine         Cloud9am           Cloud3pm         

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [62]:
# removing columns with more than 20% missing values and segregatting cat and num variables
data_cat = data[['RainToday','WindGustDir','WindDir9am','WindDir3pm']]
data.drop(columns=['Evaporation','Sunshine','Cloud9am','Cloud3pm'],axis=1,inplace=True)
data.drop(columns=['RainToday','WindGustDir','WindDir9am','WindDir3pm'],axis=1,inplace=True)

In [63]:
# filling the missing data of numeric variables with mean
data['MinTemp'].fillna(data['MinTemp'].mean(),inplace=True)
data['MaxTemp'].fillna(data['MaxTemp'].mean(),inplace=True)
data['Rainfall'].fillna(data['Rainfall'].mean(),inplace=True)
data['WindGustSpeed'].fillna(data['WindGustSpeed'].mean(),inplace=True)
data['WindSpeed9am'].fillna(data['WindSpeed9am'].mean(),inplace=True)
data['WindSpeed3pm'].fillna(data['WindSpeed3pm'].mean(),inplace=True)
data['Humidity9am'].fillna(data['Humidity9am'].mean(),inplace=True)
data['Humidity3pm'].fillna(data['Humidity3pm'].mean(),inplace=True)
data['Pressure9am'].fillna(data['Pressure9am'].mean(),inplace=True)
data['Pressure3pm'].fillna(data['Pressure3pm'].mean(),inplace=True)
data['Temp9am'].fillna(data['Temp9am'].mean(),inplace=True)
data['Temp3pm'].fillna(data['Temp3pm'].mean(),inplace=True)

In [64]:
# loading the names of categorical columns
cat_names = data_cat.columns

In [65]:
# intializing the simple imputer for missing categorical values
import numpy as np
from sklearn.impute import SimpleImputer
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [66]:
# fitting and transforming the missing data
data_cat = imp_mode.fit_transform(data_cat)

In [67]:
# converting array to dataframe
data_cat = pd.DataFrame(data_cat,columns=cat_names)

In [68]:
# concatinating the categorical and numeric data
data = pd.concat([data,data_cat],axis=1)

In [69]:
data.isnull().sum()

Date                0
Location            0
MinTemp             0
MaxTemp             0
Rainfall            0
WindGustSpeed       0
WindSpeed9am        0
WindSpeed3pm        0
Humidity9am         0
Humidity3pm         0
Pressure9am         0
Pressure3pm         0
Temp9am             0
Temp3pm             0
RainTomorrow     3267
RainToday           0
WindGustDir         0
WindDir9am          0
WindDir3pm          0
dtype: int64

In [70]:
data.shape

(145460, 19)

In [71]:
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainTomorrow,RainToday,WindGustDir,WindDir9am,WindDir3pm
0,2008-12-01,Delhi,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,No,No,W,W,WNW
1,2008-12-02,Delhi,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,No,No,WNW,NNW,WSW
2,2008-12-03,Delhi,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,No,No,WSW,W,WSW
3,2008-12-04,Delhi,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,No,No,NE,SE,E
4,2008-12-05,Delhi,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,No,No,W,ENE,NW


In [72]:
#importing the labelencoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [73]:
#fitting and transforming the categorical data
data['Location'] = le.fit_transform(data['Location'])
data['RainToday'] = le.fit_transform(data['RainToday'])
data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
data['WindDir9am'] = le.fit_transform(data['WindDir9am'])
data['WindDir3pm'] = le.fit_transform(data['WindDir3pm'])

In [74]:
# spliiting the date column into year,month,day
data[["year", "month", "day"]] = data["Date"].str.split("-", expand = True)

In [75]:
# removing the main column
data.drop(['Date'],axis=1,inplace=True)

In [76]:
data.head()

,Location,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,...,Temp9am,Temp3pm,RainTomorrow,RainToday,WindGustDir,WindDir9am,WindDir3pm,year,month,day
0,14,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,...,16.9,21.8,No,0,13,13,14,2008,12,01
1,14,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,...,17.2,24.3,No,0,14,6,15,2008,12,02
2,14,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,...,21.0,23.2,No,0,15,13,15,2008,12,03
3,14,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,...,18.1,26.5,No,0,4,9,0,2008,12,04
4,14,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,...,17.8,29.7,No,0,13,1,7,2008,12,05


In [77]:
data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

In [78]:
data.head()

,Location,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,...,Temp9am,Temp3pm,RainTomorrow,RainToday,WindGustDir,WindDir9am,WindDir3pm,year,month,day
0,14,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,...,16.9,21.8,0,0,13,13,14,2008,12,01
1,14,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,...,17.2,24.3,0,0,14,6,15,2008,12,02
2,14,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,...,21.0,23.2,0,0,15,13,15,2008,12,03
3,14,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,...,18.1,26.5,0,0,4,9,0,2008,12,04
4,14,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,...,17.8,29.7,0,0,13,1,7,2008,12,05


In [79]:
data.dropna(axis=0,how='any',inplace=True)

In [80]:
data.shape

(145460, 21)

In [81]:
y = data['RainTomorrow']
x = data.drop('RainTomorrow',axis=1)

In [82]:
names = x.columns

In [83]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [84]:
x = sc.fit_transform(x)

In [85]:
x = pd.DataFrame(x,columns=names)

In [86]:
x.head()

,Location,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,WindGustDir,WindDir9am,WindDir3pm,year,month,day
0,-0.719411,0.189447,-0.045336,-0.210072,0.302234,0.672222,0.612323,0.112395,-1.442965,-1.478788,-1.223882,-0.014053,0.017023,-0.529795,1.045228,1.328766,1.366458,-1.879575,1.634076,-1.672844
1,-0.719411,-0.753101,0.265043,-0.281650,0.302234,-1.133438,0.382874,-1.319609,-1.296417,-1.047782,-1.118839,0.032464,0.381985,-0.529795,1.258262,-0.221338,1.586813,-1.879575,1.634076,-1.559140
2,-0.719411,0.110901,0.349692,-0.281650,0.454694,0.559368,0.841771,-1.637832,-1.052171,-1.493650,-0.983784,0.621685,0.221402,-0.529795,1.471296,1.328766,1.586813,-1.879575,1.634076,-1.445436
3,-0.719411,-0.470337,0.674180,-0.281650,-1.222364,-0.343462,-1.108541,-1.266572,-1.736061,-0.007422,-0.368534,0.172017,0.703151,-0.529795,-0.872075,0.442992,-1.718521,-1.879575,1.634076,-1.331732
4,-0.719411,0.833521,1.280830,-0.162353,0.073544,-0.794877,0.153426,0.695804,-0.905623,-1.018057,-1.388949,0.125499,1.170302,-0.529795,1.045228,-1.328556,-0.176032,-1.879575,1.634076,-1.218028


In [87]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: RainTomorrow, dtype: int32

In [88]:
from sklearn import model_selection

In [89]:
x_train,x_test,y_train,y_test = model_selection.train_test_split(x,y,test_size =0.2,random_state =0)

In [90]:
#Models intilization of the models
XGBoost = xgboost.XGBRFClassifier()
Rand_forest = ensemble.RandomForestClassifier()
svm = svm.SVC()
Dtree = tree.DecisionTreeClassifier()
GBM = ensemble.GradientBoostingClassifier()
log = linear_model.LogisticRegression()

In [91]:
# fitting the model
XGBoost.fit(x_train,y_train)
Rand_forest.fit(x_train,y_train)
svm.fit(x_train,y_train)
Dtree.fit(x_train,y_train)
GBM.fit(x_train,y_train)
log.fit(x_train,y_train)

LogisticRegression()

In [92]:
# predicting the unknown values
p1 = XGBoost.predict(x_train)
p2 = Rand_forest.predict(x_train)
p3 = svm.predict(x_train)
p4 = Dtree.predict(x_train)
p5 = GBM.predict(x_train)
p6 = log.predict(x_train)

In [93]:
#checking the accuraccy score
print("xgboost:",metrics.accuracy_score(y_train,p1))
print("Rand_forest:",metrics.accuracy_score(y_train,p2))
print("svm:",metrics.accuracy_score(y_train,p3))
print("Dtree:",metrics.accuracy_score(y_train,p4))
print("GBM:",metrics.accuracy_score(y_train,p5))
print("log:",metrics.accuracy_score(y_train,p6))

xgboost: 0.8335453045510793
Rand_forest: 0.9999484394335213
svm: 0.8439605389797883
Dtree: 1.0
GBM: 0.8406606627251478
log: 0.8200020624226592


In [94]:
# predicting the test unknown values
t1 = XGBoost.predict(x_test)
t2 = Rand_forest.predict(x_test)
t3 = svm.predict(x_test)
t4 = Dtree.predict(x_test)
t5 = GBM.predict(x_test)
t6 = log.predict(x_test)

In [95]:
print("xgboost:",metrics.accuracy_score(y_test,t1))
print("Rand_forest:",metrics.accuracy_score(y_test,t2))
print("svm:",metrics.accuracy_score(y_test,t3))
print("Dtree:",metrics.accuracy_score(y_test,t4))
print("GBM:",metrics.accuracy_score(y_test,t5))
print("log:",metrics.accuracy_score(y_test,t6))

xgboost: 0.8308125945277052
Rand_forest: 0.8432558779045786
svm: 0.8357967826206517
Dtree: 0.7689399147531968
GBM: 0.8383404372336037
log: 0.8210504606077272


In [96]:
model = ensemble.RandomForestClassifier()

In [97]:
parameters = {"max_features":["log2","sqrt"],
              'n_estimators': [5,10,15,20,25],
               'criterion':['gini','entropy']}

In [99]:
Grid_search = model_selection.GridSearchCV(estimator=model,param_grid=parameters,cv=10)

In [100]:
Grid_search.fit(x_train,y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['log2', 'sqrt'],
                         'n_estimators': [5, 10, 15, 20, 25]})

In [101]:
Grid_search.best_params_  # best combination,which gives better accuracy

{'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 25}

In [102]:
model = ensemble.RandomForestClassifier(criterion='entropy',max_features='sqrt',n_estimators = 25)

In [104]:
model.fit(x_train,y_train)
y_tp= model.predict(x_train)
metrics.accuracy_score(y_train,y_tp)

0.9981610064622577

In [107]:
y_pred= model.predict(x_test)
metrics.accuracy_score(y_test,y_pred)

0.8407809707135983

In [108]:
metrics.confusion_matrix(y_test,y_pred)

array([[21027,  1017,    38],
       [ 3157,  3159,    50],
       [  275,    95,   274]], dtype=int64)

In [109]:
import pickle

In [110]:
pickle.dump(model,open('rainfall.pkl','wb'))
pickle.dump(le,open('encoder.pkl','wb'))
pickle.dump(imp_mode,open('impter.pkl','wb'))